In [1]:
# Project for Human Capital Supply and Demand
# Importing libraries
import pandas as pd
import numpy as np

In [2]:
# reading Excel file
Headcount = pd.read_excel('Demandv1.1.xlsx', 'Headcount',header=0)
Headcount = Headcount[Headcount['Status'] == 'Bench']
Headcount = Headcount.reset_index(drop = True)
Demand_LY = pd.read_excel('Demandv1.1.xlsx', 'Demand Trend Last year',header=0)

In [3]:
Demand_LY['Month DD Raised'].unique()

array(['May', 'October', 'March', 'June', 'August', 'February', 'January',
       'December'], dtype=object)

In [4]:
Supply = Headcount[Headcount['Status'] == 'Bench']

In [5]:
Supply_Smry = pd.Series(Supply['SkillList'].value_counts())

In [6]:
Supply_Smry = pd.DataFrame(Supply.pivot_table(index = 'SkillList', columns = 'Status', 
                                                 aggfunc={'Status':np.count_nonzero}))

In [7]:
#SampleData = pd.merge(Supply_Smry,Demand_Smry, left_index=True, right_index=True)

In [8]:
#SampleData['Hire'] = SampleData['No. of FTE Request Raised'] - SampleData[SampleData.columns[0]]

In [9]:
#SampleData[SampleData['Hire'] < 0].sum()

In [10]:
'''for i in range(SampleData.shape[0]):
    #print(i)
    if SampleData.iloc[i,2] < 0 :
        #print(i)
        SampleData.iloc[i,2] = 0'''

'for i in range(SampleData.shape[0]):\n    #print(i)\n    if SampleData.iloc[i,2] < 0 :\n        #print(i)\n        SampleData.iloc[i,2] = 0'

In [11]:
Demand_LY = Demand_LY.sort_values(['Month DD Raised','SkillList','Location'])

In [12]:
# Sort Demand for Last Year File by Month
rule = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8,
       'September':9, 'October':10, 'November':11, 'December':12}
Demand_LY['Month'] = Demand_LY['Month DD Raised']
Demand_LY['Month'] = Demand_LY['Month'].replace(rule)
Demand_LY = Demand_LY.sort_values(['Month','SkillList','No. of FTE Request Raised','Location'])
Demand_LY = Demand_LY.reset_index(drop = True)

Headcount = Headcount.sort_values(['SkillList','Location'])
Headcount = Headcount.reset_index(drop = True)

In [13]:
def Generate_Demand(i,Demand_LY) :
    Skill = Demand_LY.ix[i,'SkillList']
    Requirement = Demand_LY.loc[i,'No. of FTE Request Raised']
    Location = Demand_LY.ix[0,'Location']
    Month = Demand_LY.ix[0,'Month DD Raised']

    #print('-----Demand:-----')
    #print('Month:',Month)
    #print('Skill:' ,Skill) 
    #print('Requirement:', Requirement)
    #print('Location:',Location)
    return Skill,Requirement,Location,Month

In [14]:
def Generate_Supply(Skill,Headcount):
    Supply = Headcount[Headcount['Status'] == 'Bench']
    Bench = Supply[Supply['SkillList'] == Skill]
    #print('-----Bench Available:-----')
    #print('Bench:', Bench.shape[0]), print(pd.DataFrame(Bench['Location']).reset_index(drop = True))
    #print('Hire:',Hire)
    Bench = Check_SubsetSkill_Match(Demand_LY,Headcount,Skill,Supply,Bench)
    Hire = Requirement-Bench.shape[0]
    return Hire,Bench

In [15]:
rows = []
def Check_SubsetSkill_Match(Demand_LY,Headcount,Skill,Supply,Bench):
    #Supply.reset_index(drop = True)
    #print('----Checking for Subset----',Skill)
    Supply = Supply[Supply['SkillList'] != Skill]
    for i in Supply.index:
        Head_Skill = Supply.loc[i,'SkillList']
        lst1 = list(Head_Skill.split(','))  #list(Supply.loc[i,'SkillList'].split(','))
        lst2 = list(Skill.split(','))
        if (set(lst2).issubset(lst1)) == 1 :
            Bench = Bench.append(Supply.loc[i,])
    return Bench

In [16]:
Headcount['Relocated City'] = Headcount['Location']
Headcount['Client Date of Joining'] = 'NA'

In [17]:
def Allocate_Bench(Bench,Location,Month,Headcount):
    # Allocate from Bench
    rows = Bench.index
    Headcount.loc[rows,'Status'] = 'Billable'
    Headcount.loc[rows,'Relocated City'] = Location
    Headcount.loc[rows,'Client Date of Joining'] = Month
    return Headcount

def New_Hire(Month,Skill,Location,Headcount):
    # New Hire Append to Headcount
    new_hire = pd.DataFrame([['Region','Employee Code','Last Name',Month,
                         'Designation','Billable','Market Unit',
                         Skill,Location,Location,Month]], columns = Headcount.columns)
    for j in range(Hire) :
        Headcount = Headcount.append(new_hire, ignore_index=True)
    return Headcount

In [18]:
import random
def New_Headcount(Headcount):
    Count_of_Attrition = round(.2*(Headcount.shape[0]))
    rows_Bench = list(Headcount[Headcount['Status'] == 'Bench'].index)
    if Count_of_Attrition > len(Headcount[Headcount['Status'] == 'Bench']):
        Count_of_Attrition = len(Headcount[Headcount['Status'] == 'Bench'])
    Attrition = random.sample(rows_Bench, Count_of_Attrition)
    Headcount = Headcount.drop(Attrition)
    return Headcount

In [19]:
for i in range(Demand_LY.shape[0]):
    Skill,Requirement,Location,Month = Generate_Demand(i,Demand_LY)
    Hire,Bench = Generate_Supply(Skill,Headcount)
    if Bench.shape[0] > 0 :
        Headcount = Allocate_Bench(Bench,Location,Month,Headcount)
    if Hire > 0 :
        Headcount = New_Hire(Month,Skill,Location,Headcount)
    Headcount = New_Headcount(Headcount)
    #input()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [20]:
Headcount.shape

(2543, 11)

In [21]:
Headcount

,Region,Employee Code,Last Name,Local Date of Joining,Designation,Status,Market Unit,SkillList,Location,Relocated City,Client Date of Joining
0,IN,59989,TIWARI,2012-01-22 00:00:00,PORTFOLIO MANAGER,Billable,NaN,"(D) Capital Markets Sector,Financial Services",Bhubaneswar,Chennai,January
1,UK,39107,TIWARI,2014-08-06 00:00:00,MANAGER,Billable,NaN,"(D) Capital Markets Sector,Financial Services",Hyderabad,Chennai,January
2,US,85982,GORIJALA,2010-03-15 00:00:00,PORTFOLIO MANAGER,Billable,BANK,.Net with MVC,Noida,Chennai,January
3,US,36375,KHARBANDA,2012-07-30 00:00:00,MANAGER,Billable,INS,".Net with MVC,SQL Server 2012",Pune,Chennai,January
4,UK,33948,SHARMA,2015-08-05 00:00:00,MANAGER,Billable,INS,Ab Initio,Bangalore,Chennai,January
5,IN,91011,SAKHARE,2017-09-01 00:00:00,MANAGER,Billable,INS,Ab Initio,Hyderabad,Chennai,January
6,SG,27223,RASIDI,2000-08-07 00:00:00,SENIOR CONSULTANT,Billable,INS,Ab Initio,Pune,Chennai,January
7,IN,24313,PILLAI,2002-10-15 00:00:00,SENIOR MANAGER,Billable,BANK,"Java,API Integrations- REST, SOAP , Powershell",Gurgaon,Chennai,January
8,US,75827,JOHRI,2006-08-28 00:00:00,PORTFOLIO MANAGER,Billable,GP,"Spring Batch,API Integrations- REST, SOAP , Po...",Hartford,Chennai,January
9,Region,Employee Code,Last Name,January,Designation,Billable,Market Unit,"(D) Capital Markets Sector,Financial Services",Chennai,Chennai,January
